# Lab | API wrappers - Create your collection of songs & audio features

Antonio Montilla

In [12]:
#importing libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
import pandas as pd
from pandas import json_normalize

In [3]:
#connecting with Spotify API
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [5]:
#secrets_dict

In [44]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

#### (1) Playlist: https://open.spotify.com/playlist/5xqpyfZyS1DVysoevdVyEn

In [7]:
#5xqpyfZyS1DVysoevdVyEn
playlist = sp.user_playlist_tracks("spotify", "5xqpyfZyS1DVysoevdVyEn")

In [9]:
#creating function to get all tracks
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items'] #this will add only 100 tracks
    while results['next']!=None: #ie, while there another next
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [10]:
#applying function on playlist id 5xqpyfZyS1DVysoevdVyEn
all_tracks = get_playlist_tracks("5xqpyfZyS1DVysoevdVyEn")
len(all_tracks)

10991

In [13]:
tracks2 = json_normalize(all_tracks)

In [14]:
#creating a function to expand the tracks2 output:
def expand_list_dict2(row):
    df = json_normalize(row['track.artists'])
    df['song_id'] = row['track.id']
    return df

tracks2['artists_dfs'] = tracks2.apply(expand_list_dict2, axis=1)

In [15]:
#now creating a new dataframe with all these artists
artist_df2 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks2['artists_dfs']:
    #display(val)
    artist_df2 = pd.concat([artist_df2, mini_df], axis=0)
    
artist_df2

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/1xU878Z1QtB...,1xU878Z1QtBldR7ru9owdU,Phoenix,artist,spotify:artist:1xU878Z1QtBldR7ru9owdU,https://open.spotify.com/artist/1xU878Z1QtBldR...,3AA8xNhDC0MpqwkGX3EP5V
0,https://api.spotify.com/v1/artists/2oWyxHNcxMc...,2oWyxHNcxMcrvNIHMX2apR,Radio 4,artist,spotify:artist:2oWyxHNcxMcrvNIHMX2apR,https://open.spotify.com/artist/2oWyxHNcxMcrvN...,2i8lRhmZNKaM1ypHwRybuD
0,https://api.spotify.com/v1/artists/2UyhhfLOvfL...,2UyhhfLOvfLs7ZhzsAaNC3,Mount Sims,artist,spotify:artist:2UyhhfLOvfLs7ZhzsAaNC3,https://open.spotify.com/artist/2UyhhfLOvfLs7Z...,4flxgPC0426CEeSrzQIic0
0,https://api.spotify.com/v1/artists/1gkSl4XpHIH...,1gkSl4XpHIHI4I1WQbfXOE,Peaches,artist,spotify:artist:1gkSl4XpHIHI4I1WQbfXOE,https://open.spotify.com/artist/1gkSl4XpHIHI4I...,1XHFob24QklIXtLRopKirJ
0,https://api.spotify.com/v1/artists/3GpAL7oEFD3...,3GpAL7oEFD37IJDOOiirqy,Zongamin,artist,spotify:artist:3GpAL7oEFD37IJDOOiirqy,https://open.spotify.com/artist/3GpAL7oEFD37IJ...,4JB847zlgViLq8tJIzRsZy
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/4HpHDJqNFP3...,4HpHDJqNFP3XsGRy0SKTq4,SLEEP RADIO,artist,spotify:artist:4HpHDJqNFP3XsGRy0SKTq4,https://open.spotify.com/artist/4HpHDJqNFP3XsG...,1v6vHkDHBjm4OHUzeOKber
0,https://api.spotify.com/v1/artists/2sriRQRt36D...,2sriRQRt36DnfHtD68zZlj,Dorian Concept,artist,spotify:artist:2sriRQRt36DnfHtD68zZlj,https://open.spotify.com/artist/2sriRQRt36DnfH...,5TN8n4LQS8Or2WEnU56pXs
0,https://api.spotify.com/v1/artists/54QMjE4toDf...,54QMjE4toDfiCryzYWCpXX,Metronomy,artist,spotify:artist:54QMjE4toDfiCryzYWCpXX,https://open.spotify.com/artist/54QMjE4toDfiCr...,6MEzr1cmAJi3FXBjXVXMnm
0,https://api.spotify.com/v1/artists/5wFXmYsg3KF...,5wFXmYsg3KFJ8BDsQudJ4f,Manchester Orchestra,artist,spotify:artist:5wFXmYsg3KFJ8BDsQudJ4f,https://open.spotify.com/artist/5wFXmYsg3KFJ8B...,6ovSjbBKatyx9KvWsxkAZo


In [16]:
#now merging the two dataframes, keeping only the fields we need
df_merged2 = pd.merge(left=tracks2,
                    right=artist_df2,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_final2 = df_merged2[['track.name', 'name', 'song_id']]
df_final2

,track.name,name,song_id
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy
...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo


In [17]:
#Step (2): obtaining audio features
chunks = [(i, i+100) for i in range(0, len(df_final2), 100)]
audio_features_list = []
for chunk in chunks:
    id_list100 = df_final2['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

12681

In [18]:
audio_features_df = json_normalize(audio_features_list)

In [19]:
audio_features_df.drop_duplicates(inplace=True)

In [20]:
df_w_audio_ft1 = pd.merge(left=df_final2,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft1 
#for the clustering model, you may need to drop non-numeric columns
#I will repeat this process for more playlists and at the end I will join all data into a df_final

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,...,0.0646,0.836,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,...,0.0711,0.843,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,...,0.0700,0.948,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,...,0.0707,0.944,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,...,0.1700,0.704,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,Hate Is an Attractive Force,SLEEP RADIO,1v6vHkDHBjm4OHUzeOKber,0.759,0.121,7,-25.346,0,0.0284,0.41000,...,0.0905,0.523,90.029,audio_features,1v6vHkDHBjm4OHUzeOKber,spotify:track:1v6vHkDHBjm4OHUzeOKber,https://api.spotify.com/v1/tracks/1v6vHkDHBjm4...,https://api.spotify.com/v1/audio-analysis/1v6v...,170667,4
12677,Promises,Dorian Concept,5TN8n4LQS8Or2WEnU56pXs,0.621,0.675,10,-9.347,0,0.0685,0.38500,...,0.3470,0.323,124.056,audio_features,5TN8n4LQS8Or2WEnU56pXs,spotify:track:5TN8n4LQS8Or2WEnU56pXs,https://api.spotify.com/v1/tracks/5TN8n4LQS8Or...,https://api.spotify.com/v1/audio-analysis/5TN8...,237064,1
12678,16 Beat,Metronomy,6MEzr1cmAJi3FXBjXVXMnm,0.781,0.812,7,-11.866,1,0.0468,0.01860,...,0.0716,0.785,115.001,audio_features,6MEzr1cmAJi3FXBjXVXMnm,spotify:track:6MEzr1cmAJi3FXBjXVXMnm,https://api.spotify.com/v1/tracks/6MEzr1cmAJi3...,https://api.spotify.com/v1/audio-analysis/6MEz...,196027,4
12679,Amplified In The Silence,Manchester Orchestra,6ovSjbBKatyx9KvWsxkAZo,0.557,0.117,1,-13.880,1,0.0344,0.95000,...,0.1120,0.122,111.742,audio_features,6ovSjbBKatyx9KvWsxkAZo,spotify:track:6ovSjbBKatyx9KvWsxkAZo,https://api.spotify.com/v1/tracks/6ovSjbBKatyx...,https://api.spotify.com/v1/audio-analysis/6ovS...,223107,4


#### (2) Playlist: https://open.spotify.com/playlist/1hMzceeWw7QiI6vaBkcEJO

In [21]:
#1hMzceeWw7QiI6vaBkcEJO
playlist = sp.user_playlist_tracks("spotify", "1hMzceeWw7QiI6vaBkcEJO")

In [22]:
#applying get_playlist_tracks function on playlist id 1hMzceeWw7QiI6vaBkcEJO
all_tracks = get_playlist_tracks("1hMzceeWw7QiI6vaBkcEJO")
len(all_tracks)

10000

In [23]:
#json_normalizing all_tracks
tracks2 = json_normalize(all_tracks)

In [24]:
#applying function expand_list to tracks2
tracks2['artists_dfs'] = tracks2.apply(expand_list_dict2, axis=1)

In [25]:
#now creating a new dataframe with all these artists
artist_df2 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks2['artists_dfs']:
    #display(val)
    artist_df2 = pd.concat([artist_df2, mini_df], axis=0)
    
artist_df2

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/7abqAQhqbQV...,7abqAQhqbQVO2WgB2twSQL,MassiveMusic,artist,spotify:artist:7abqAQhqbQVO2WgB2twSQL,https://open.spotify.com/artist/7abqAQhqbQVO2W...,6uDlwsguxFDgbakvPyhChp
0,https://api.spotify.com/v1/artists/63GCe948jqV...,63GCe948jqVvNvFB0DmuGB,Hoosky,artist,spotify:artist:63GCe948jqVvNvFB0DmuGB,https://open.spotify.com/artist/63GCe948jqVvNv...,00LvjMnpznr4MZFSA8x9sA
1,https://api.spotify.com/v1/artists/2EMmqQQmszs...,2EMmqQQmszsCXfVfMRibOQ,La Fine Equipe,artist,spotify:artist:2EMmqQQmszsCXfVfMRibOQ,https://open.spotify.com/artist/2EMmqQQmszsCXf...,00LvjMnpznr4MZFSA8x9sA
2,https://api.spotify.com/v1/artists/1mM65AjhdrT...,1mM65AjhdrTa1eDExLKRsu,oOgo,artist,spotify:artist:1mM65AjhdrTa1eDExLKRsu,https://open.spotify.com/artist/1mM65AjhdrTa1e...,00LvjMnpznr4MZFSA8x9sA
3,https://api.spotify.com/v1/artists/0CJDBEjJQYG...,0CJDBEjJQYGcyC12FI1L0b,Chomsk',artist,spotify:artist:0CJDBEjJQYGcyC12FI1L0b,https://open.spotify.com/artist/0CJDBEjJQYGcyC...,00LvjMnpznr4MZFSA8x9sA
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/2I36EjIVz3v...,2I36EjIVz3vDfROgj1MfZ3,Boozoo Bajou,artist,spotify:artist:2I36EjIVz3vDfROgj1MfZ3,https://open.spotify.com/artist/2I36EjIVz3vDfR...,1PNaGC2ihDmVCldSce119E
1,https://api.spotify.com/v1/artists/1stlWvYSCm3...,1stlWvYSCm3sSEIzdKBSeY,Joe Dukie,artist,spotify:artist:1stlWvYSCm3sSEIzdKBSeY,https://open.spotify.com/artist/1stlWvYSCm3sSE...,1PNaGC2ihDmVCldSce119E
2,https://api.spotify.com/v1/artists/5N6EzjkOoyA...,5N6EzjkOoyABhNZJggeXi6,Mousse T.,artist,spotify:artist:5N6EzjkOoyABhNZJggeXi6,https://open.spotify.com/artist/5N6EzjkOoyABhN...,1PNaGC2ihDmVCldSce119E
0,https://api.spotify.com/v1/artists/2oofDquWt9t...,2oofDquWt9tMCETKAHmhlG,Mocky,artist,spotify:artist:2oofDquWt9tMCETKAHmhlG,https://open.spotify.com/artist/2oofDquWt9tMCE...,2887tlqmjY4WcyQ3Dy6dZy


In [26]:
#now merging the two dataframes, keeping only the fields we need
df_merged2 = pd.merge(left=tracks2,
                    right=artist_df2,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_final2 = df_merged2[['track.name', 'name', 'song_id']]
df_final2

,track.name,name,song_id
0,This Magic Moment - Crisp.nl Version,MassiveMusic,6uDlwsguxFDgbakvPyhChp
1,00h00,Hoosky,00LvjMnpznr4MZFSA8x9sA
2,00h00,La Fine Equipe,00LvjMnpznr4MZFSA8x9sA
3,00h00,oOgo,00LvjMnpznr4MZFSA8x9sA
4,00h00,Chomsk',00LvjMnpznr4MZFSA8x9sA
...,...,...,...
11841,Take It Slow - Mousse T. Remix,Boozoo Bajou,1PNaGC2ihDmVCldSce119E
11842,Take It Slow - Mousse T. Remix,Joe Dukie,1PNaGC2ihDmVCldSce119E
11843,Take It Slow - Mousse T. Remix,Mousse T.,1PNaGC2ihDmVCldSce119E
11844,Birds of a Feather,Mocky,2887tlqmjY4WcyQ3Dy6dZy


In [27]:
#Step (2): obtaining audio features
chunks = [(i, i+100) for i in range(0, len(df_final2), 100)]
audio_features_list = []
for chunk in chunks:
    id_list100 = df_final2['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

11846

In [28]:
audio_features_df = json_normalize(audio_features_list)

In [29]:
audio_features_df.drop_duplicates(inplace=True)

In [30]:
df_w_audio_ft2 = pd.merge(left=df_final2,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft2 

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,This Magic Moment - Crisp.nl Version,MassiveMusic,6uDlwsguxFDgbakvPyhChp,0.823,0.671,11,-6.437,1,0.0592,0.55100,...,0.3540,0.662,125.052,audio_features,6uDlwsguxFDgbakvPyhChp,spotify:track:6uDlwsguxFDgbakvPyhChp,https://api.spotify.com/v1/tracks/6uDlwsguxFDg...,https://api.spotify.com/v1/audio-analysis/6uDl...,188697,4
1,00h00,Hoosky,00LvjMnpznr4MZFSA8x9sA,0.375,0.606,4,-9.383,0,0.1970,0.33900,...,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
2,00h00,La Fine Equipe,00LvjMnpznr4MZFSA8x9sA,0.375,0.606,4,-9.383,0,0.1970,0.33900,...,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
3,00h00,oOgo,00LvjMnpznr4MZFSA8x9sA,0.375,0.606,4,-9.383,0,0.1970,0.33900,...,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
4,00h00,Chomsk',00LvjMnpznr4MZFSA8x9sA,0.375,0.606,4,-9.383,0,0.1970,0.33900,...,0.4240,0.883,91.564,audio_features,00LvjMnpznr4MZFSA8x9sA,spotify:track:00LvjMnpznr4MZFSA8x9sA,https://api.spotify.com/v1/tracks/00LvjMnpznr4...,https://api.spotify.com/v1/audio-analysis/00Lv...,51373,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11841,Take It Slow - Mousse T. Remix,Boozoo Bajou,1PNaGC2ihDmVCldSce119E,0.684,0.504,9,-8.913,1,0.0801,0.09720,...,0.3360,0.740,86.226,audio_features,1PNaGC2ihDmVCldSce119E,spotify:track:1PNaGC2ihDmVCldSce119E,https://api.spotify.com/v1/tracks/1PNaGC2ihDmV...,https://api.spotify.com/v1/audio-analysis/1PNa...,216601,4
11842,Take It Slow - Mousse T. Remix,Joe Dukie,1PNaGC2ihDmVCldSce119E,0.684,0.504,9,-8.913,1,0.0801,0.09720,...,0.3360,0.740,86.226,audio_features,1PNaGC2ihDmVCldSce119E,spotify:track:1PNaGC2ihDmVCldSce119E,https://api.spotify.com/v1/tracks/1PNaGC2ihDmV...,https://api.spotify.com/v1/audio-analysis/1PNa...,216601,4
11843,Take It Slow - Mousse T. Remix,Mousse T.,1PNaGC2ihDmVCldSce119E,0.684,0.504,9,-8.913,1,0.0801,0.09720,...,0.3360,0.740,86.226,audio_features,1PNaGC2ihDmVCldSce119E,spotify:track:1PNaGC2ihDmVCldSce119E,https://api.spotify.com/v1/tracks/1PNaGC2ihDmV...,https://api.spotify.com/v1/audio-analysis/1PNa...,216601,4
11844,Birds of a Feather,Mocky,2887tlqmjY4WcyQ3Dy6dZy,0.677,0.410,9,-10.598,0,0.0400,0.59100,...,0.0813,0.515,81.924,audio_features,2887tlqmjY4WcyQ3Dy6dZy,spotify:track:2887tlqmjY4WcyQ3Dy6dZy,https://api.spotify.com/v1/tracks/2887tlqmjY4W...,https://api.spotify.com/v1/audio-analysis/2887...,274013,4


#### (3) Playlist: https://open.spotify.com/playlist/4rnleEAOdmFAbRcNCgZMpY

In [55]:
#4rnleEAOdmFAbRcNCgZMpY
playlist = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [56]:
#applying get_playlist_tracks function on playlist id 4rnleEAOdmFAbRcNCgZMpY
all_tracks = get_playlist_tracks("4rnleEAOdmFAbRcNCgZMpY")
len(all_tracks)

5295

In [57]:
#json_normalizing all_tracks
tracks2 = json_normalize(all_tracks)

In [58]:
#applying function expand_list to tracks2
tracks2['artists_dfs'] = tracks2.apply(expand_list_dict2, axis=1)

In [59]:
#now creating a new dataframe with all these artists
artist_df2 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri'])
for mini_df in tracks2['artists_dfs']:
    #display(val)
    artist_df2 = pd.concat([artist_df2, mini_df], axis=0)
    
artist_df2

,href,id,name,type,uri,external_urls.spotify,song_id
0,https://api.spotify.com/v1/artists/2FXC3k01G6G...,2FXC3k01G6Gw61bmprjgqS,Hozier,artist,spotify:artist:2FXC3k01G6Gw61bmprjgqS,https://open.spotify.com/artist/2FXC3k01G6Gw61...,7dS5EaCoMnN7DzlpT6aRn2
0,https://api.spotify.com/v1/artists/2KsP6tYLJlT...,2KsP6tYLJlTBvSUxnwlVWa,Mike Posner,artist,spotify:artist:2KsP6tYLJlTBvSUxnwlVWa,https://open.spotify.com/artist/2KsP6tYLJlTBvS...,2V4bv1fNWfTcyRJKmej6Sj
1,https://api.spotify.com/v1/artists/1Bo8Afb2Qbj...,1Bo8Afb2Qbjs4x6kJHyjle,Gigamesh,artist,spotify:artist:1Bo8Afb2Qbjs4x6kJHyjle,https://open.spotify.com/artist/1Bo8Afb2Qbjs4x...,2V4bv1fNWfTcyRJKmej6Sj
0,https://api.spotify.com/v1/artists/4V8LLVI7Pba...,4V8LLVI7PbaPR0K2TGSxFF,"Tyler, The Creator",artist,spotify:artist:4V8LLVI7PbaPR0K2TGSxFF,https://open.spotify.com/artist/4V8LLVI7PbaPR0...,7KA4W4McWYRpgf0fWsJZWB
1,https://api.spotify.com/v1/artists/1U1el3k54Vv...,1U1el3k54VvEUzo3ybLPlM,Kali Uchis,artist,spotify:artist:1U1el3k54VvEUzo3ybLPlM,https://open.spotify.com/artist/1U1el3k54VvEUz...,7KA4W4McWYRpgf0fWsJZWB
...,...,...,...,...,...,...,...
0,https://api.spotify.com/v1/artists/1McMsnEElTh...,1McMsnEElThX1knmY4oliG,Olivia Rodrigo,artist,spotify:artist:1McMsnEElThX1knmY4oliG,https://open.spotify.com/artist/1McMsnEElThX1k...,61KpQadow081I2AsbeLcsb
0,https://api.spotify.com/v1/artists/6veh5zbFpm3...,6veh5zbFpm31XsPdjBgPER,BIA,artist,spotify:artist:6veh5zbFpm31XsPdjBgPER,https://open.spotify.com/artist/6veh5zbFpm31Xs...,5yorXJWdBan1Vlh116ZtQ7
0,https://api.spotify.com/v1/artists/3PyJHH2wyfQ...,3PyJHH2wyfQK3WZrk9rpmP,Ashnikko,artist,spotify:artist:3PyJHH2wyfQK3WZrk9rpmP,https://open.spotify.com/artist/3PyJHH2wyfQK3W...,11ZulcYY4lowvcQm4oe3VJ
1,https://api.spotify.com/v1/artists/6lay1nwbE6h...,6lay1nwbE6hTx1jivysUAL,Princess Nokia,artist,spotify:artist:6lay1nwbE6hTx1jivysUAL,https://open.spotify.com/artist/6lay1nwbE6hTx1...,11ZulcYY4lowvcQm4oe3VJ


In [60]:
#now merging the two dataframes, keeping only the fields we need
df_merged2 = pd.merge(left=tracks2,
                    right=artist_df2,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
df_final2 = df_merged2[['track.name', 'name', 'song_id']]
df_final2

,track.name,name,song_id
0,Take Me To Church,Hozier,7dS5EaCoMnN7DzlpT6aRn2
1,Cooler Than Me - Single Mix,Mike Posner,2V4bv1fNWfTcyRJKmej6Sj
2,Cooler Than Me - Single Mix,Gigamesh,2V4bv1fNWfTcyRJKmej6Sj
3,See You Again (feat. Kali Uchis),"Tyler, The Creator",7KA4W4McWYRpgf0fWsJZWB
4,See You Again (feat. Kali Uchis),Kali Uchis,7KA4W4McWYRpgf0fWsJZWB
...,...,...,...
7744,deja vu,Olivia Rodrigo,61KpQadow081I2AsbeLcsb
7745,WHOLE LOTTA MONEY,BIA,5yorXJWdBan1Vlh116ZtQ7
7746,Slumber Party (feat. Princess Nokia),Ashnikko,11ZulcYY4lowvcQm4oe3VJ
7747,Slumber Party (feat. Princess Nokia),Princess Nokia,11ZulcYY4lowvcQm4oe3VJ


In [61]:
#Step (2): obtaining audio features
chunks = [(i, i+100) for i in range(0, len(df_final2), 100)]
audio_features_list = []
for chunk in chunks:
    id_list100 = df_final2['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

7749

In [62]:
audio_features_df = json_normalize(audio_features_list)

In [63]:
audio_features_df.drop_duplicates(inplace=True)

In [64]:
df_w_audio_ft3 = pd.merge(left=df_final2,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft3 

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Take Me To Church,Hozier,7dS5EaCoMnN7DzlpT6aRn2,0.566,0.664,4,-5.303,0,0.0464,0.63400,...,0.116,0.437,128.945,audio_features,7dS5EaCoMnN7DzlpT6aRn2,spotify:track:7dS5EaCoMnN7DzlpT6aRn2,https://api.spotify.com/v1/tracks/7dS5EaCoMnN7...,https://api.spotify.com/v1/audio-analysis/7dS5...,241688,4
1,Cooler Than Me - Single Mix,Mike Posner,2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0474,0.17900,...,0.689,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
2,Cooler Than Me - Single Mix,Gigamesh,2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0474,0.17900,...,0.689,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
3,See You Again (feat. Kali Uchis),"Tyler, The Creator",7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,0.37100,...,0.109,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
4,See You Again (feat. Kali Uchis),Kali Uchis,7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,0.37100,...,0.109,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744,deja vu,Olivia Rodrigo,61KpQadow081I2AsbeLcsb,0.439,0.610,9,-7.236,1,0.1160,0.59300,...,0.341,0.172,181.088,audio_features,61KpQadow081I2AsbeLcsb,spotify:track:61KpQadow081I2AsbeLcsb,https://api.spotify.com/v1/tracks/61KpQadow081...,https://api.spotify.com/v1/audio-analysis/61Kp...,215508,4
7745,WHOLE LOTTA MONEY,BIA,5yorXJWdBan1Vlh116ZtQ7,0.897,0.371,1,-5.019,1,0.3680,0.09040,...,0.325,0.441,81.008,audio_features,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,https://api.spotify.com/v1/tracks/5yorXJWdBan1...,https://api.spotify.com/v1/audio-analysis/5yor...,156005,4
7746,Slumber Party (feat. Princess Nokia),Ashnikko,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.101,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4
7747,Slumber Party (feat. Princess Nokia),Princess Nokia,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.101,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4


#### Combining all dataframes to create songs_total

In [70]:
songs_total = pd.concat([df_w_audio_ft1, df_w_audio_ft2, df_w_audio_ft3], axis=0, ignore_index=True)

In [71]:
songs_total

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,...,0.0646,0.836,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,...,0.0711,0.843,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,...,0.0700,0.948,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,...,0.0707,0.944,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,...,0.1700,0.704,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32271,deja vu,Olivia Rodrigo,61KpQadow081I2AsbeLcsb,0.439,0.610,9,-7.236,1,0.1160,0.59300,...,0.3410,0.172,181.088,audio_features,61KpQadow081I2AsbeLcsb,spotify:track:61KpQadow081I2AsbeLcsb,https://api.spotify.com/v1/tracks/61KpQadow081...,https://api.spotify.com/v1/audio-analysis/61Kp...,215508,4
32272,WHOLE LOTTA MONEY,BIA,5yorXJWdBan1Vlh116ZtQ7,0.897,0.371,1,-5.019,1,0.3680,0.09040,...,0.3250,0.441,81.008,audio_features,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,https://api.spotify.com/v1/tracks/5yorXJWdBan1...,https://api.spotify.com/v1/audio-analysis/5yor...,156005,4
32273,Slumber Party (feat. Princess Nokia),Ashnikko,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.1010,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4
32274,Slumber Party (feat. Princess Nokia),Princess Nokia,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.1010,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4


In [72]:
#cleaning potential duplicates
songs_total.drop_duplicates(inplace=True)

In [73]:
songs_total

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,If I Ever Feel Better,Phoenix,3AA8xNhDC0MpqwkGX3EP5V,0.755,0.553,6,-9.449,0,0.1070,0.04920,...,0.0646,0.836,119.700,audio_features,3AA8xNhDC0MpqwkGX3EP5V,spotify:track:3AA8xNhDC0MpqwkGX3EP5V,https://api.spotify.com/v1/tracks/3AA8xNhDC0Mp...,https://api.spotify.com/v1/audio-analysis/3AA8...,266600,4
1,Dance To The Underground,Radio 4,2i8lRhmZNKaM1ypHwRybuD,0.667,0.667,7,-6.267,1,0.0388,0.00013,...,0.0711,0.843,130.321,audio_features,2i8lRhmZNKaM1ypHwRybuD,spotify:track:2i8lRhmZNKaM1ypHwRybuD,https://api.spotify.com/v1/tracks/2i8lRhmZNKaM...,https://api.spotify.com/v1/audio-analysis/2i8l...,243213,4
2,How We Do,Mount Sims,4flxgPC0426CEeSrzQIic0,0.826,0.601,10,-3.966,0,0.0515,0.19500,...,0.0700,0.948,126.971,audio_features,4flxgPC0426CEeSrzQIic0,spotify:track:4flxgPC0426CEeSrzQIic0,https://api.spotify.com/v1/tracks/4flxgPC0426C...,https://api.spotify.com/v1/audio-analysis/4flx...,234000,4
3,Fuck the Pain Away,Peaches,1XHFob24QklIXtLRopKirJ,0.905,0.945,4,-2.630,1,0.1370,0.03540,...,0.0707,0.944,131.597,audio_features,1XHFob24QklIXtLRopKirJ,spotify:track:1XHFob24QklIXtLRopKirJ,https://api.spotify.com/v1/tracks/1XHFob24QklI...,https://api.spotify.com/v1/audio-analysis/1XHF...,250693,4
4,Street Surgery 2,Zongamin,4JB847zlgViLq8tJIzRsZy,0.797,0.536,11,-8.267,0,0.2660,0.03150,...,0.1700,0.704,102.881,audio_features,4JB847zlgViLq8tJIzRsZy,spotify:track:4JB847zlgViLq8tJIzRsZy,https://api.spotify.com/v1/tracks/4JB847zlgViL...,https://api.spotify.com/v1/audio-analysis/4JB8...,224800,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32271,deja vu,Olivia Rodrigo,61KpQadow081I2AsbeLcsb,0.439,0.610,9,-7.236,1,0.1160,0.59300,...,0.3410,0.172,181.088,audio_features,61KpQadow081I2AsbeLcsb,spotify:track:61KpQadow081I2AsbeLcsb,https://api.spotify.com/v1/tracks/61KpQadow081...,https://api.spotify.com/v1/audio-analysis/61Kp...,215508,4
32272,WHOLE LOTTA MONEY,BIA,5yorXJWdBan1Vlh116ZtQ7,0.897,0.371,1,-5.019,1,0.3680,0.09040,...,0.3250,0.441,81.008,audio_features,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,https://api.spotify.com/v1/tracks/5yorXJWdBan1...,https://api.spotify.com/v1/audio-analysis/5yor...,156005,4
32273,Slumber Party (feat. Princess Nokia),Ashnikko,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.1010,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4
32274,Slumber Party (feat. Princess Nokia),Princess Nokia,11ZulcYY4lowvcQm4oe3VJ,0.964,0.398,11,-8.981,0,0.0795,0.00151,...,0.1010,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405,4


In [74]:
#saving the file into a new csv, so I do not need to run everything again
songs_total.to_csv('songs_total.csv', index=False)